<a href="https://colab.research.google.com/github/KhadkaBibek/prompt_engineering-/blob/main/prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

|Name                        | Task  Done |
|---------------------------|------------|
|Bibek Khadka(500216129)   | Used Prompt Engineering Techinique like zero-shot, Few-shot, Chain of thought|

| Prompting Technique                     | Average ROUGE-L | Average BERT Score |
|-----------------------------------------|-----------------|---------------------|
| Zero-Shot Prompting                     | 0.0231         | 0.7914|
| Few-Shot Prompting                      | 0.1043          | 0.8382             |
| CoT                                     | 0.0333          | 0.8637
          

In [57]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pre.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,human,gpt,cleaned_human,cleaned_gpt
0,identity_0,"I""ve been feeling so sad and overwhelmed latel...","Hey there, I""m here to listen and support you....",feel sad overwhelm late work stress,hey listen support sound like work late tell w...
1,identity_0,"I recently got a promotion at work, which I th...",I can understand how it can be overwhelming wh...,recent got work thought would taken toll menta...,understand overwhelm face higher expect okay a...
2,identity_0,"Well, the workload has increased significantly...","It sounds like you""re dealing with a lot of pr...",well workload find hard maintain stay late fee...,sound like your deal lot perform succeed cruci...
3,identity_0,"I""ve been trying to prioritize my tasks and de...","It""s great to hear that you""re already impleme...",task also start break help stress feel like ma...,great hear your implement help progress take t...
4,identity_0,"You""re right. I haven""t really opened up about...","It""s completely normal to feel that way, but r...",your right open supervisor guess afraid appear...,normal feel way ask support strength weak star...
...,...,...,...,...,...
807080,identity_99469,"She""s been incredibly patient and understandin...",The fact that your girlfriend cares deeply abo...,she patient understand want support tell take ...,fact girlfriend care want work testament bond ...
807081,identity_99469,"I think it stems from unresolved trauma, Alex....","Trauma can often be a factor in addiction, Cha...",think stem trauma loss year ago turn way numb ...,trauma often factor addict connect loss addict...
807082,identity_99469,"It has affected my work, friendships, and over...","It""s completely understandable that addiction ...",affect work friendship feel like lost way addi...,understand addict setback aspect life setback ...
807083,identity_99469,I want to break free from the grip of addictio...,"Charlie, your determination to overcome addict...",want break free grip addict want regain contro...,addict rebuild life journey profession support...


In [2]:
!pip install groq
import os
from typing import Dict, List
from groq import Groq
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 2.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
os.environ['GROQ_API_KEY'] = "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"

# Models
DEFAULT_MODEL = 'llama-3.1-70b-versatile'


# CLIENTS TO INTERACT WITH GROQ
client = Groq()


In [4]:
# This function sends a series of messages (a conversation) to a chat API and retrieves the assistant's response.
#Sends a list of messages (formatted using the above functions) to a chatbot API
def chat_completion(
    message : List[Dict],
    model : str = DEFAULT_MODEL,
    temperature : float = 0.6,
    top_p: float = 0.9,

) -> str:
    try:
        response = client.chat.completions.create(
          messages = message,
          model = model,
          temperature = temperature,
          top_p = top_p,
    )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return "Error generating response."




In [30]:
## Zero-Shot Prompting
#A simplified wrapper for chat_completion for zero-shot learning.
def completion(
    prompt:str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,

) -> str:
    return chat_completion([user(prompt)],model = model,temperature = temperature,top_p = top_p,)


def user_and_print(prompt:str):
    print(f'==============\n{prompt}\n==============')
    response = completion(prompt, DEFAULT_MODEL)
    return response




In [31]:
#These functions format messages for the chatbot API by assigning roles (user, assistant, or system) and associating them with the corresponding content.
## Message Formatting
def user(content: str):
    return {"role": "user", "content": content}

def assistant(content: str):
    return {"role": "assistant", "content": content}

def system(content: str):
    return {"role": "system", "content": content}

In [32]:
user_and_print(df['cleaned_human'][5])

thank understand remind face ill gather soon need find healthier


"It sounds like you're feeling unwell and need to find healthier options.  Here are some suggestions:\n\n1. **Stay hydrated**: Drink plenty of water and other fluids to help your body recover.\n2. **Rest**: Give your body the rest it needs to fight off the illness.\n3. **Healthy foods**: Opt for nutrient-rich foods like fruits, vegetables, whole grains, and lean proteins.\n4. **Avoid triggers**: Identify and avoid triggers that can make your illness worse, such as certain foods or environmental factors.\n5. **Seek medical attention**: If your symptoms persist or worsen, consult with a healthcare professional for proper diagnosis and treatment.\n\nSome healthier food options to consider:\n\n* Soups (like chicken noodle or vegetable-based soups)\n* Herbal teas (like peppermint or ginger)\n* Fresh fruits (like bananas or berries)\n* Steamed vegetables (like broccoli or carrots)\n* Lean proteins (like chicken or fish)\n\nRemember to take care of yourself and prioritize your health."

In [33]:
# Print some of the text generated using zero shot technique.
sample = df.sample(2)
human_text = sample['cleaned_human'].tolist()
gpt_text = sample['cleaned_gpt'].tolist()
# Now generate text using Llama model
llama_response = []
for i in human_text:

  llama_response.append(user_and_print(i))



well thing bring joy help distract mind start paint vibrant color stroke take long walk find surround reconnect relationship
thank willing help late feel health doubt make question


In [34]:
llama_response


["It sounds like you're looking for ways to bring joy and positivity into your life.  Here are some suggestions based on what you've mentioned:\n\n1. **Creative Expression**: Painting with vibrant colors can be a great way to express yourself and tap into your creativity. You can try using different techniques, such as watercolor, acrylic, or even digital painting.\n2. **Nature Walks**: Taking a long walk in nature can be incredibly uplifting and help clear your mind. Being surrounded by trees, flowers, and wildlife can help you feel more connected to the world around you.\n3. **Reconnecting with Others**: Nurturing relationships with loved ones can bring joy and a sense of belonging. Try reaching out to a friend or family member you haven't spoken to in a while, or plan a fun activity with someone you care about.\n4. **Mindfulness**: Practicing mindfulness can help distract your mind from worries and stress. Try incorporating mindfulness exercises into your daily routine, such as medi

In [37]:
from rouge_score import rouge_scorer
from bert_score import score as bert_score

def compute_rougeL_bert_score(gpt_outputs, llama_responses):
    """
    Compute ROUGE-L and BERT Score for model outputs.
    """
    # Initialize Scorer
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    rouge_scores = []
    bert_scores = []

    for gpt, llama in zip(gpt_outputs, llama_responses):
        # Compute ROUGE-L
        rouge = rouge_scorer_instance.score(gpt, llama)['rougeL'].fmeasure
        rouge_scores.append(rouge)

        # Compute BERT Score
        _, _, bert = bert_score([llama], [gpt], lang="en")
        bert_scores.append(bert.mean().item())

    # Compute Average Scores
    avg_rouge = sum(rouge_scores) / len(rouge_scores)
    avg_bert = sum(bert_scores) / len(bert_scores)

    return {
        "average_rougeL": avg_rouge,
        "average_bert_score": avg_bert,
    }

In [38]:

compute_rougeL_bert_score(gpt_text[:2], llama_response[:2] )

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'average_rougeL': 0.02310052600818235,
 'average_bert_score': 0.7914449870586395}

In [40]:
# Generate few-short list
# Creates a few-shot prompt list by randomly sampling n examples from a dataset.

def random_sample(n:int, df:pd.DataFrame):
  few_shot = []
  random_data = df.sample(n)
  human_list = random_data['cleaned_human']
  gpt_list = random_data['cleaned_gpt']

  # Add system message
  few_shot.append(system("You are a chatbot. For each message, return a response based on user input."))

  # Add user and assistant examples
  for i,j in zip(human_list, gpt_list):
    few_shot.append(user(i))
    few_shot.append(assistant(j))
  return few_shot



In [41]:
# Generate response

def generate_response(few_shot:List[Dict], text:str):
  "Generate response from chat model"
  print(f"Type of few_shot: {type(few_shot)}")
  few_shot.append(user(text))
  response = chat_completion(few_shot)
  return response

In [42]:
# Evaluation Function
def evaluate_response(human_input:str, gpt_output :str, llama_response:str) -> Dict:

  """
 Evaluate the Llama response against the GPT response using semantic similarity.
 """
  model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
  embeddings = model.encode([gpt_output,llama_response])
  similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
  return {'human_input':human_input, 'gpt_output':gpt_output, 'llama_response':llama_response,'similarity_score':similarity_score  }



In [43]:
def few_shot_pipeline(df:pd.DataFrame, sample_size:int, test_cases : int ):
  """
  Run the few-shot prompt engineering pipeline and evaluate the response for multiple test cases
  """
  result = []
  few_shot_list = random_sample(sample_size, df)

  test_samples = df.sample(test_cases)
  for _, row in test_samples.iterrows():
    human_input = row['cleaned_human']
    gpt_output = row['cleaned_gpt']

    #Generate Llama response
    start_time = time.time()
    llama_response = generate_response(few_shot_list, human_input)
    elapsed_time = time.time() - start_time

    print(f'Llama response : {llama_response}')
    print(f'Response_time:{elapsed_time}')

    #Evaluate and store result
    evaluation_results = evaluate_response(human_input, gpt_output, llama_response)
    evaluation_results['response_time'] = elapsed_time
    result.append(evaluation_results)
    print(f"Evaluation Results: {evaluation_results}\n")

  return result



In [44]:
# Run pipeline with 10 - shots promt and 2 test cases
result = few_shot_pipeline(df, sample_size = 10,  test_cases = 2)
result


Type of few_shot: <class 'list'>
Llama response : beautiful perspective hold onto hope inner voice remind inner critic voice not define worth inner strength guide
Response_time:0.3933720588684082
Evaluation Results: {'human_input': 'time start love success begin doubt question person inner critic part still hold onto hope', 'gpt_output': 'share thought take strength confront inner critic want know love matter may suggest let part hold onto hope work toward', 'llama_response': 'beautiful perspective hold onto hope inner voice remind inner critic voice not define worth inner strength guide', 'similarity_score': 0.6784609, 'response_time': 0.3933720588684082}

Type of few_shot: <class 'list'>
Llama response : you're welcome glad could be there support you through tough time know it's not easy share pain but willing listen help you through it
Response_time:46.92926049232483
Evaluation Results: {'human_input': 'thank kind support help moment think area support willing listen pain journey', 

[{'human_input': 'time start love success begin doubt question person inner critic part still hold onto hope',
  'gpt_output': 'share thought take strength confront inner critic want know love matter may suggest let part hold onto hope work toward',
  'llama_response': 'beautiful perspective hold onto hope inner voice remind inner critic voice not define worth inner strength guide',
  'similarity_score': 0.6784609,
  'response_time': 0.3933720588684082},
 {'human_input': 'thank kind support help moment think area support willing listen pain journey',
  'gpt_output': 'honor hold space need talk seek heal heartbreak journey time support love one find way back inner',
  'llama_response': "you're welcome glad could be there support you through tough time know it's not easy share pain but willing listen help you through it",
  'similarity_score': 0.40353116,
  'response_time': 46.92926049232483}]

In [48]:
output_from_gpt =  [ a ['gpt_output'] for a in result ]
output_from_llama = [ a['llama_response'] for a in result ]

In [49]:
compute_rougeL_bert_score(output_from_gpt, output_from_llama)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'average_rougeL': 0.10433689503456944,
 'average_bert_score': 0.8382487595081329}

In [50]:
# Generate few shot list with coT examples
def random_sample_cot(df:pd.DataFrame, n:int) -> List[Dict]:
  few_shot = []
  random_data = df.sample(n)
  human_list = random_data['cleaned_human']
  gpt_list = random_data['cleaned_gpt']

  # ADD system message
  few_shot.append(system("You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries."))

  # Add user and assistant examples.
  for human, gpt in zip(human_list, gpt_list):
    few_shot.append(user(human))
    few_shot.append(assistant(gpt))
  return few_shot


In [51]:
def few_shot_with_cot(df:pd.DataFrame, sample_size:int, test_cases:int) -> List[Dict]:
    """
    Run the few-shot prompt engineering pipeline and evaluate responses for multiple test cases.
    """
    results = []
    few_shot_list = random_sample_cot(df, sample_size)
    print(few_shot_list)
    test_samples = df.sample(test_cases)
    for _,row in test_samples.iterrows():
      human_input = row['cleaned_human']
      gpt_output = row['cleaned_gpt']


      # Generate Llama response
      start_time = time.time()
      llama_response = generate_response(few_shot_list, human_input)
      elapsed_time = time.time() - start_time

    # Evaluate and store result
      evaluation_result = evaluate_response(human_input, gpt_output, llama_response)
      evaluation_result["response_time"] = elapsed_time
      results.append(evaluation_result)


    return results


In [52]:
result = few_shot_with_cot(df, 5 ,2)
result

[{'role': 'system', 'content': 'You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries.'}, {'role': 'user', 'content': 'moment feel much lighter arm wait start see life thank time understand'}, {'role': 'assistant', 'content': 'assist delight hear your feel lighter take step toward need talk wonder day'}, {'role': 'user', 'content': 'actual feel comfort talk ill talk see come plan workload hope feel'}, {'role': 'assistant', 'content': 'that great step lighten burden also foster teamwork need effect concern like address'}, {'role': 'user', 'content': 'often find journal write thought help process go like pain also find comfort look old photograph cherish share'}, {'role': 'assistant', 'content': 'journal power tool heal allow express space reflect grief journey would like question cope grief'}, {'role': 'user', 'content': 'guess one thing like work build support system much turn thing get difficult want learn let without feel like burden'}, {'ro

[{'human_input': 'surround support understand journey also differ break free restrict feel',
  'gpt_output': 'that wonder great hear your take step seek support other feel moment',
  'llama_response': 'empathy understand journey free space allow break free restrict feel empower surround support also encourage take time focus',
  'similarity_score': 0.4950331,
  'response_time': 0.322648286819458},
 {'human_input': nan,
  'gpt_output': 'understand feel hurt spoken partner',
  'llama_response': 'Error generating response.',
  'similarity_score': -0.05630818,
  'response_time': 0.10092544555664062}]

In [54]:
output_from_gpt =  [ a ['gpt_output'] for a in result ]
output_from_llama = [ a['llama_response'] for a in result ]

In [55]:
compute_rougeL_bert_score(output_from_gpt, output_from_llama)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'average_rougeL': 0.03333333333333333,
 'average_bert_score': 0.8637712597846985}

Here, we used BERT Score as a key metric to evaluate semantic understanding, essential for assessing chatbot response quality.

| Prompting Technique                     | Average ROUGE-L | Average BERT Score |
|-----------------------------------------|-----------------|---------------------|
| Zero-Shot Prompting                     | 0.0231         | 0.7914|
| Few-Shot Prompting                      | 0.1043          | 0.8382             |
| CoT                                     | 0.0333          | 0.8637
            
